# A notebook to read a CSV template with Site/Individual/Skeleton Element info and load it to elab

Please enter file name in the following cell, and what is the delimiter in that file

In [1]:
CSVfilename="/Users/pierrespc/Documents/PostDocPasteur/aDNA/GeneralInfo/2022-01_RawExcellFromArchaesologists/ineLabWithPicturesSunsistence_Mendez/2022-03-21_Mendez_43.169.txt"
delimiterFile="\t"

Please enter the one-line file where your token is saved in the following cell

In [2]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"

Now preparing all required python libs

In [3]:
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}
params={}




Reading the data. SkelDict is to make sure thta if eLab configuration change we just need to change here and not the excel template. 

In [4]:

DICT={"Site":{
        "Name":"Site",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        "Unit":"fixed_unit",
        "Main geographic region":"Site_Main Geographic Region",
        "Country":"Site_Country",
        "Province / Region":"Site_Province / Region",
        "Locality":"Site_Locality",
        "Latitude":"Site_LatChanged",
        "Longitude":"Site_LongChanged",
        "Site type":"Site_Site type",
        "Pictures":"None",
        "parentSampleID":"None"
    },

    "Individual":{
        "Name":"Individual",
        #"Description":"Individual_Archaeologist Observations",
        "Description":"None",
        "Note":"None",
        "Amount":"fixed_1",
        #"Unit":"fixed_Unit | pcs",
        "Unit":"fixed_unit",
        "parentSampleID":"Site",
        "Archaeologist ID":"Individual_Archaeologist ID",
        "Archaeologist group":"Individual_Archaeologist group",
        "Site Name":"Site",
        "Date":"Individual_Date",
        "Datation method":"Individual_Datation method",
        "Subsistence Strategy": "Individual_Subsistence Strategy",
        "Age":"Individual_Age",
        "Gender":"Individual_Gender",
        "Pictures":"None",
        "Linked individuals":"None"
    },

    "Skeleton Element":{"Name":"Skeleton Element",
         "From Individual":"Individual",
         "Description":"Skeleton Element_description",
         "Note":"None",
         "Amount":"fixed_1",
         #"Unit":"fixed_Unit | pcs",
         "Unit":"fixed_unit",
         "parentSampleID":"Individual",
         "Archaeologist sample ID":"Skeleton Element_Archaeologist sample ID",
         "Pictures Labelling":"Skeleton Element_Pictures Labelling",
         "Pictures Drilling":"None",
         "Bone type":"Skeleton Element_Bone type",
         "Skeleton element":"Skeleton Element_Skeleton element",
         "Exportation Permit Number":"Skeleton Element_Exportation Permit Number",
         "Observation Labelling":"Skeleton Element_Observation Labelling",
         "Observation Scanning":"Skeleton Element_Observation Scanning",
         "Observation Drilling":"None",
         "Scanned":"Skeleton Element_Scanned"

         
    }
}




Table=pandas.read_csv(CSVfilename,delimiter=delimiterFile)

Table

,Site,Site_Main Geographic Region,Site_Country,Site_Province / Region,Site_Locality,Site_Latitude,Site_Longitude,Site_Site type,Individual,Individual_Archaeologist ID,...,Individual_Archaeologist Observations,Skeleton Element,Skeleton Element_Archaeologist sample ID,Skeleton Element_description,Skeleton Element_Pictures Labelling,Skeleton Element_Skeleton element,Skeleton Element_Exportation Permit Number,Skeleton Element_Observation Labelling,Skeleton Element_Scanned,Skeleton Element_Observation Scanning
0,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0001,04Ca012-P-9-1,...,NaN,CH0001.1,04Ca012-P-9-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured,NaN,NaN
1,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0002,04Ca012-P-9-2,...,NaN,CH0002.1,04Ca012-P-9-2,Disturbado (posible osario),/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured,NaN,NaN
2,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0003,04Ca012-P-26-1,...,NaN,CH0003.1,04Ca012-P-26-1,Dec�bito lateral derecho,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN,NaN,NaN
3,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0004,04Ca012-P-33-1,...,NaN,CH0004.1,04Ca012-P-33-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN,NaN,NaN
4,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0005,04Ca012-P-50-1,...,NaN,CH0005.1,04Ca012-P-50-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,Looks very fragile. fisured. Tip of the root b...,NaN,NaN
5,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0006,04Ca012-P-51-1,...,NaN,CH0006.1,04Ca012-P-51-1,Dec�bito dorsal,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured. No emanel. Very dark,NaN,NaN
6,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0007,04Ca012-P-51-2,...,NaN,CH0007.1,04Ca012-P-51-2,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,covered by dark layer. Fisured. One little bro...,NaN,NaN
7,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0008,04Ca012-P-57-1,...,NaN,CH0008.1,04Ca012-P-57-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN,NaN,NaN
8,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0009,04Ca012-P-57-2,...,NaN,CH0009.1,04Ca012-P-57-2,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,broken root tips. Aparent pulp,NaN,NaN
9,Agua Dulce,Costa chilena,Chile,NaN,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0010,04Ca012-P-91-1,...,NaN,CH0010.1,04Ca012-P-91-1,Dec�bito dorsal,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,only emanel,NaN,NaN


We remove all columns that only have null value

In [5]:
colDrop=[]
for col in Table.keys():
    if Table[col].isnull().all():
        print(col+" only empty values--> we remove")
        colDrop.append(col)
print(colDrop)
Table=Table.drop(labels=colDrop,axis=1)
Table

Site_Province / Region only empty values--> we remove
Individual_Date only empty values--> we remove
Individual_Datation method only empty values--> we remove
Individual_Archaeologist Observations only empty values--> we remove
Skeleton Element_Scanned only empty values--> we remove
Skeleton Element_Observation Scanning only empty values--> we remove
['Site_Province / Region', 'Individual_Date', 'Individual_Datation method', 'Individual_Archaeologist Observations', 'Skeleton Element_Scanned', 'Skeleton Element_Observation Scanning']


,Site,Site_Main Geographic Region,Site_Country,Site_Locality,Site_Latitude,Site_Longitude,Site_Site type,Individual,Individual_Archaeologist ID,Individual_Archaeologist group,Individual_Subsistence Strategy,Individual_Age,Individual_Gender,Skeleton Element,Skeleton Element_Archaeologist sample ID,Skeleton Element_description,Skeleton Element_Pictures Labelling,Skeleton Element_Skeleton element,Skeleton Element_Exportation Permit Number,Skeleton Element_Observation Labelling
0,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0001,04Ca012-P-9-1,C�sar M�ndez Catherine Westfall,Cazador recolector,Probable adulto joven,Pendiente,CH0001.1,04Ca012-P-9-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured
1,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0002,04Ca012-P-9-2,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Probable adulto joven,Pendiente,CH0002.1,04Ca012-P-9-2,Disturbado (posible osario),/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured
2,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0003,04Ca012-P-26-1,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Posible sub adulto,Pendiente,CH0003.1,04Ca012-P-26-1,Dec�bito lateral derecho,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN
3,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0004,04Ca012-P-33-1,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Posible infante,Pendiente,CH0004.1,04Ca012-P-33-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN
4,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0005,04Ca012-P-50-1,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Posible infante,Pendiente,CH0005.1,04Ca012-P-50-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,Looks very fragile. fisured. Tip of the root b...
5,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0006,04Ca012-P-51-1,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Posible adulto,Pendiente,CH0006.1,04Ca012-P-51-1,Dec�bito dorsal,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,fisured. No emanel. Very dark
6,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0007,04Ca012-P-51-2,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Posible adulto maduro,Pendiente,CH0007.1,04Ca012-P-51-2,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,covered by dark layer. Fisured. One little bro...
7,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0008,04Ca012-P-57-1,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Probable adulto joven,Pendiente,CH0008.1,04Ca012-P-57-1,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,NaN
8,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0009,04Ca012-P-57-2,C�sar M�ndez Catherine Westfall,Cazador recolector costero,Probable infante,Pendiente,CH0009.1,04Ca012-P-57-2,Dec�bito lateral izquierdo,/pasteur/entites/metapaleo/Research/ERC-projec...,Diente,/pasteur/entites/metapaleo/Research/ERC-projec...,broken root tips. Aparent pulp
9,Agua Dulce,Costa chilena,Chile,Huentelauqu�n,"31d31'10,10''S","71d33'59,98''O",Cementerio,CH0010,04Ca012-P-91-1,C�

Make "Skeleton Element_Bone type"

In [6]:
print(Table)

                      Site Site_Main Geographic Region Site_Country  \
0               Agua Dulce               Costa chilena        Chile   
1               Agua Dulce               Costa chilena        Chile   
2               Agua Dulce               Costa chilena        Chile   
3               Agua Dulce               Costa chilena        Chile   
4               Agua Dulce               Costa chilena        Chile   
5               Agua Dulce               Costa chilena        Chile   
6               Agua Dulce               Costa chilena        Chile   
7               Agua Dulce               Costa chilena        Chile   
8               Agua Dulce               Costa chilena        Chile   
9               Agua Dulce               Costa chilena        Chile   
10              Agua Dulce               Costa chilena        Chile   
11              Agua Dulce               Costa chilena        Chile   
12              Agua Dulce               Costa chilena        Chile   
13    

In [7]:
Table.keys()

Index(['Site', 'Site_Main Geographic Region', 'Site_Country', 'Site_Locality',
       'Site_Latitude', 'Site_Longitude', 'Site_Site type', 'Individual',
       'Individual_Archaeologist ID', 'Individual_Archaeologist group',
       'Individual_Subsistence Strategy', 'Individual_Age',
       'Individual_Gender', 'Skeleton Element',
       'Skeleton Element_Archaeologist sample ID',
       'Skeleton Element_description', 'Skeleton Element_Pictures Labelling',
       'Skeleton Element_Skeleton element',
       'Skeleton Element_Exportation Permit Number',
       'Skeleton Element_Observation Labelling'],
      dtype='object')

In [8]:
import itertools

def defineBoneTypefunction(ele):
    l1=list(range(1,9)) * 8
    l2=list(itertools.chain.from_iterable(itertools.repeat(x, 8) for x in list(range(1,9))))
    l3=[]
    for i in list(range(0,len(l1))):
        l3.append(str(l1[i])+"."+str(l2[i]))


    toothElements=["tooth",
               "diente",
               "molar",
               "mx",
               "42 inf. 1° derecho",
               " m ",
               " md",
               "incisive",
               "incisivo",
               "incisor",
               "canino",
               "canine",
               l3,
               "1.1 o 2.1",
               "1.5 o 2.5",
                "1mid",
                "1mii",
                "1pmid",
                "1pmii",
                "2iid",
                "2iii",
                "2isi",
                "2mid",
                "2mii",
                "2pmid",
                "2pmii",
                "2pmsd",
                "2pmsi",
                "2isd",
                "2isi",
                "2minf.",
                "3mid",
                "3mii",
                "3minf."]
    
    calculusElements=["calculus",
                      "from 1isd",
                      "from 1isi",
                      "from 1pmii",
                      "from 2iii",
                      "from 2pmii",
                      "from 3mii",
                      "from cid"]

    petrousElements=["petrous",
                 "petroso",
                 "petrozo"]

    undefinedElements=["??"]
    otherElements=["rib",
               "fibula",
               "matoideo",
               "metatarsal",
               "humer",
               "humerus",
               "lumbar",
               "craneo",
               "vertebra",
               "tibia",
               "radio",
               "phalanx",
               "phalange",
               "longbone",
               "femur",
               "metacarpo",
               "metatarsus",
               "tarseano",
               "hueso",
               "femur",
                "calota",
              "tarso"]

    noHumanElements=["valva"]    
    
    lowerEle=ele.lower()
    if bool([ttt for ttt in toothElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("Tooth")        
    elif bool([ttt for ttt in calculusElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("Dental Calculus")
    elif bool([ttt for ttt in petrousElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("Petrous")
    elif bool([ttt for ttt in otherElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("Other Bone")
    elif bool([ttt for ttt in undefinedElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("Undefined")
    elif bool([ttt for ttt in noHumanElements if(lowerEle in ttt or str(ttt) in lowerEle)]):
        return("non Human")
    else:
        print(ele+" issue...")


Table[DICT['Skeleton Element']['Bone type']]=Table[DICT['Skeleton Element']['Skeleton element']].apply(defineBoneTypefunction)
Table[DICT['Skeleton Element']['Bone type']]

0     Tooth
1     Tooth
2     Tooth
3     Tooth
4     Tooth
5     Tooth
6     Tooth
7     Tooth
8     Tooth
9     Tooth
10    Tooth
11    Tooth
12    Tooth
13    Tooth
14    Tooth
15    Tooth
16    Tooth
17    Tooth
18    Tooth
19    Tooth
20    Tooth
21    Tooth
22    Tooth
23    Tooth
24    Tooth
25    Tooth
26    Tooth
27    Tooth
28    Tooth
29    Tooth
30    Tooth
31    Tooth
32    Tooth
33    Tooth
34    Tooth
35    Tooth
36    Tooth
37    Tooth
38    Tooth
39    Tooth
Name: Skeleton Element_Bone type, dtype: object

Format longitud and latitude 

In [9]:
###change coordinates
def changeCOORfunction(coord):
    stuf="º"
    stuf2="°"
    #if coord is None or numpy.isnan(coord):
    if coord is None:
        #return([None,None,None,None])
        return(None)
                      
    coord=format(coord)
    coordFed=coord
    #print(coord)
    #####HERE CHANGE ALL WEIRD CHARACTERS THAT CAN COME ON THE WAY
    coord=coord.replace("39° 06 ́","39°06'")
    coord=coord.replace("63° 47 ́","63°47'") 
    ##change weird degrees characters
    coord=coord.replace(stuf2,stuf)
    coord=coord.replace("d",stuf)
    ##change weird minutes characters
    coord=coord.replace("``","\"")
    coord=coord.replace("´´","\"")
    coord=coord.replace("”","\"")
    coord=coord.replace("''","\"")
    coord=coord.replace("’’","\"")
    coord=coord.replace("“","\"")
    ##change weird seconds characters
    coord=coord.replace(" ´","'")
    coord=coord.replace("`","'")
    coord=coord.replace("´","'")
    coord=coord.replace("’","'")
    ###we now that it is all South and West
    coord=coord.replace(" ","")
    coord=coord.replace("S","")
    coord=coord.replace("O","")
    coord=coord.replace("W","")
    ###change decimal character
    coord=coord.replace(",",".")

    ##read degrees
    if len(coord.split(stuf)) ==2 :
        if coord.split(stuf)[1] == "":
            if stuf not in coord:
                coord=coord + stuf + "0'"
            else:
                coord=coord+"0'"
    elif len(coord.split(stuf)) == 1 :
        if stuf not in coord:
            coord=coord + stuf + "0'"
        else:
            coord=coord+"0'"
    else:
        print("splitting minute/second " + coordFed + "-->" + coord)
            
    deg=coord.split(stuf)[0]
    ###read minutes and seconds
    tmp=coord.split(stuf)[1]
    minute=tmp.split("'")[0]
    if len(tmp.split("'")) != 2:
        print( coord.split(stuf))
        print("splitting minute/second " + coordFed + "-->" + coord)
        raise()
    else:
        if tmp.split("'")[1] == "":
            sec=0
        else:
            sec=tmp.split("'")[1]
            sec=sec.replace("\"","")

    #print([coordFed,coord,deg,minute,sec])
    ####verify all read ok!
    if numpy.isnan(float(deg)):
        print("pb numerical degree" + coordFed + "-->" +coord + " (" + deg + ")")
        raise()
    if numpy.isnan(float(minute)):
        print(minute)
        print("pb numerical minute" + coordFed + "-->" +coord + " (" + minute + ")")
        raise()
    if numpy.isnan(float(sec)):
        print(sec)
        print("pb numerical sec" + coordFed + "-->" + coord + " (" + sec + ")")
        raise()
  
    deg=float(deg)
    minute=float(minute)
    sec=float(sec)
    new=deg+minute/60+sec/3600
    if(new<0):
        return(new)
    else:
        return(-new)



In [10]:
if "Site_Latitude" in Table.keys():

    Table.loc[Table['Site_Latitude'].isnull(),"Site_Latitude"]=None
    Table.loc[Table['Site_Longitude'].isnull(),"Site_Longitud"]=None
    
    
    Table.loc[Table["Site_Latitude"].apply(format).isin([ "Undefined","desconocido","nan","NA"]),"Site_Latitude"]=None
    Table.loc[Table["Site_Longitude"].apply(format).isin([ "Undefined","desconocido","nan","NA"]),"Site_Longitude"]=None

    #print(rawTab.loc[rawTab['Latitude']==""])
    Table['Site_LatChanged']=Table['Site_Latitude'].apply(changeCOORfunction)
    Table['Site_LongChanged']=Table['Site_Longitude'].apply(changeCOORfunction)

    print(Table[["Site_LatChanged","Site_Latitude","Site_LongChanged","Site_Longitude"]])

    Site_LatChanged   Site_Latitude  Site_LongChanged  Site_Longitude
0        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
1        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
2        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
3        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
4        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
5        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
6        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
7        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
8        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
9        -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
10       -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
11       -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
12       -31.519472  31d31'10,10''S        -71.566661  71d33'59,98''O
13       -31.519472 

Prepare all the eLab-API keys necessary to down and upload data

In [11]:
def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)


r = requests.get(url + "sampleTypes", headers = headers2)
if BadRequest(r,200):
    r.raise_for_status()
data = r.json()
types = {}
for typ in data.get("data"):
    types[format(typ.get("name"))] = format(typ.get("sampleTypeID"))

print(types)


FeateLab={}
for sampTY in ["Individual","Skeleton Element","Site"]:
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    FeateLab[sampTY] = {}
    for feat in ['Name','Description','Note','Amount','Unit',"parentSampleID"]:
        FeateLab[sampTY][feat] = {"ID": "notMeta"}
    for feat in data.get("data"):
        FeateLab[sampTY][format(feat.get("key"))] = { "ID":format(feat.get("sampleTypeMetaID")),
                                              "TYPE":format(feat.get("sampleDataType"))}

print(FeateLab)

{'Individual': '39466', 'Site': '39468', 'Skeleton Element': '39469', 'Extract': '39470', 'Indexed Library': '39494', 'Library pool': '39495', 'Non Indexed Library': '39556', 'Bone pellet': '39599'}
{'Individual': {'Name': {'ID': 'notMeta'}, 'Description': {'ID': 'notMeta'}, 'Note': {'ID': 'notMeta'}, 'Amount': {'ID': 'notMeta'}, 'Unit': {'ID': 'notMeta'}, 'parentSampleID': {'ID': 'notMeta'}, 'Archaeologist group': {'ID': '244121', 'TYPE': 'CHECKBOX'}, 'Archaeologist ID': {'ID': '244132', 'TYPE': 'TEXT'}, 'Site Name': {'ID': '244133', 'TYPE': 'SAMPLELINK'}, 'Date': {'ID': '244134', 'TYPE': 'TEXT'}, 'Datation method': {'ID': '244135', 'TYPE': 'TEXTAREA'}, 'Age': {'ID': '244136', 'TYPE': 'TEXT'}, 'Gender': {'ID': '244137', 'TYPE': 'COMBO'}, 'Linked individuals': {'ID': '244138', 'TYPE': 'TEXT'}, 'Pictures': {'ID': '244141', 'TYPE': 'FILE'}, 'Subsistence Strategy': {'ID': '246288', 'TYPE': 'TEXT'}}, 'Skeleton Element': {'Name': {'ID': 'notMeta'}, 'Description': {'ID': 'notMeta'}, 'Note': 

Check the columns in  Table are recognized here. If no lines in output, you are just fine.

In [12]:
for sampTY in  FeateLab.keys():
    for feat in FeateLab[sampTY].keys():
        if feat not in DICT[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN DICTIONARY")
        
    for feat in DICT[sampTY].keys():
        if feat not in FeateLab[sampTY].keys():
            exit(sampTY+": "+feat + "--> NOT IN eLAB")
        


We get all the possible values for checkboxes and dropdown features of Extracts and check our extractTable table is fine. If no lines in output, you're just fine

In [13]:
for sampTY in FeateLab.keys():
    r = requests.get(url + "sampleTypes/" + types[sampTY] + "/meta", headers = headers2)
    data = r.json()
    for feat in data.get("data"):
        key=feat.get("key")
        if key not in DICT[sampTY].values():
            continue
        if feat.get("sampleDataType") == "CHECKBOX" or feat.get("sampleDataType") == "COMBO":
            OptionELAB=feat.get("optionValues")
            if DICT[sampTY][key].startswith("fixed"):
                tabVal=DICT[sampTY][key].split("_")[1]
                if tabVal not in OptionELAB:
                    exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)
            else:
                Table.loc[Table[DICT[sampTY][key]].isnull(),DICT[sampTY][key]]="NA"
                for tabVal in Table[DICT[sampTY][key]].unique():
                    if tabVal not in OptionELAB:
                        exit(sampTY+": " + tabVal + "-- not mapped in eLab for " + key)



Now, we make the json for each extract and we upload or update in eLab!
Change the default prompt line:
- put y if you are sure you want to overwrite already loaded info in eLab, 
- put n if you are sure you want to leave already loaded info in eLab (although it doesn't match info in your table)
- put anything else if you want a case by case prompt

In [14]:
####get all registered skeleton element and extracts
registered = {}
for name in ["Skeleton Element","Individual","Site"]:
    #print(name)
    r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': types[name]})
    if BadRequest(r,200):
        r.raise_for_status()
    data = r.json()
    myList = {}
    for sam in data.get("data"):
        if format(sam.get("name")) in myList.keys():
            print(name + ": " + sam.get("name") + " duplicated")
            break
        myList[format(sam.get("name"))]=format(sam.get("sampleID"))
    registered[name] = myList



In [16]:
defaultPrompt="y"

#for sampTy in ["Site","Individual","Skeleton Element"]:
for sampTy in ["Individual","Skeleton Element"]:
    
    ###make an unique TableTYPE
    TableTYPE={}
    for fea in DICT[sampTy].values():
        if fea != "None" and not fea.startswith("fixed") and fea in Table.keys():
            TableTYPE[fea]=Table[fea]
        
    TableTYPE=pandas.DataFrame(TableTYPE).drop_duplicates()
    
    ###iterate over extracts in table
    for index,name in TableTYPE[DICT[sampTy]['Name']].items():
        print(name)
        if format(name)=="nan":
            continue
        patch=False
        ###get what is already uploaded for that extract in eLab
        if name in registered[sampTy].keys():
            patch=True
            id=registered[sampTy][name]
            r=requests.get(url + "samples/"+id, headers = headers2)
            if BadRequest(r,200):
                r.raise_for_status()
            r=r.json()
        
            ###change to lower case all the keys because API sometimes use upper, lower for different request (A MESS!)
            dataLoaded={}
            for oldkey in r:
                newkey=oldkey.lower()
                dataLoaded[newkey] = r[oldkey]
        ####prepare the Data to be loaded
        Data={}
        for fea in FeateLab[sampTy].keys():
            if FeateLab[sampTy][fea]['ID'] == "notMeta" and fea not in ["Amount","Unit"]:
                ###fixed value (from dico)
                if DICT[sampTy][fea].startswith("fixed"):
                    element=DICT[sampTy][fea].split("_")[1]
                ###WHEN NO VALUE to enter
                elif DICT[sampTy][fea]=="None" or DICT[sampTy][fea] not in TableTYPE.keys():
                    if fea == "parentSampleID":
                        element=0
                    else:
                        element="NA"
                elif fea == "parentSampleID":
                    if sampTy == "Site":
                        continue
                    element=registered[DICT[sampTy]["parentSampleID"]][TableTYPE[DICT[sampTy]["parentSampleID"]][index]]
                else:
                    element=TableTYPE[DICT[sampTy][fea]][index]
                ###check delta when patching
                if patch:
                    elementLoaded=dataLoaded[fea.lower()]
                    if format(elementLoaded) != format(element):
                        print("For "+sampTy+" "+name+", do you want to update the "+fea+" field? That is: "+format(element)+ " vs what already loaded: "+format(elementLoaded))
                        if DICT[sampTy][fea]=="None":
                            print("no update for None features")
                            prompt="n"
                        else:
                            prompt=defaultPrompt
                            while prompt not in ["y","n"]:
                                prompt = input("replace y/n??")
                        if prompt == "n":
                             element=elementLoaded
                Data[fea]=element
        ###case of updating
        if patch:
            DR=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
            if BadRequest(DR,204):
                DR.raise_for_status()
        else:
            ###case of uploading
            #print(name + "uploading")
            patch=False
            Data["sampleTypeID"]=types[sampTy]
            Data["Name"]=name
            DR=requests.post(url + "samples/", headers = headers2,data = Data)   

        ####check the Data loading was correct
        if BadRequest(DR,204):
            DR.raise_for_status()
            
        ###actualize the registered[<sample type>] list (checking we did not duplicated anything here)
        r=requests.get(url + "samples/forNames?names="+name, headers = headers2)
        if BadRequest(r,200):
            r.raise_for_status()
        data=r.json()
        sam=data.get("data")
        if len(sam)!=1:
            exit("different "+sampTy+" entries (" + str(len(sam)) + ") for name "+name)    
        else:
            sam=sam[0]
            id=str(sam.get("sampleID"))
            #print("Data OK for "+ name + " (" + id + ")")
            registered[sampTy][name]=id

        ###patch the metaData
        
        if patch:
            #print("patching meta so need to check if differences for "+name)
            MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
            if BadRequest(MDR,200):
                MDR.raise_for_status()
            data=MDR.json().get("data")
            metaLoaded={}
            for i in data:
                metaLoaded[i["key"]]=str(i["value"])
        for fea in FeateLab[sampTy].keys():

            needToPatch=False
            #MDR=requests.get(url + "samples/"+id+"/meta", headers = headers2)
            #if BadRequest(MDR,200):
            #    MDR.raise_for_status()
            ###get new element to be loaded
            if FeateLab[sampTy][fea]['ID'] != "notMeta" and FeateLab[sampTy][fea]['TYPE'] != "FILE":
                ###fixed value (from dico)
                if DICT[sampTy][fea].startswith("fixed"):
                    element=DICT[sampTy][fea].split("_")[1]
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                elif DICT[sampTy][fea]=="None":
                    element="NA"
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
                elif (fea == "From Individual" and sampTy =="Skeleton Element") or (fea == "Site Name" and sampTy =="Individual"):
                    sisi=TableTYPE[DICT[sampTy][fea]][index]
                    IDsisi=registered[DICT[sampTy][fea]][sisi]
                    element=sisi+"|"+IDsisi
                    samples={"sampleID": IDsisi,"name": sisi}
                
                    MetaData={
                        "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                        "sampleDataType": FeateLab[sampTy][fea]['TYPE'],
                        "samples": samples,
                        "key": fea,
                        "value": element
                    }
                elif DICT[sampTy][fea] in TableTYPE.keys():
                    element=TableTYPE[DICT[sampTy][fea]][index]
                    if format(element)=="nan" or format(element)=="" or format(element)==" ":
                        element="NA"
                    MetaData={"key": fea,
                              "sampleTypeMetaID": int(FeateLab[sampTy][fea]['ID']),
                              "value": element,
                              "sampleDataType": FeateLab[sampTy][fea]['TYPE']}
            
                ###check if this is a new entry or not
                if patch:
                    ###check if new element is similar to what already loaded
                    if fea not in metaLoaded.keys(): 
                        needToPatch=True
                    elif metaLoaded[fea] != str(element):
                        print("difference for " + name + "(feature: " + fea + ") " + str(element) + " vs loaded : " + metaLoaded[fea])
                        if DICT[sampTy][fea]=="None":
                            print("no update for None features")
                            prompt="n"
                        else:
                            prompt=defaultPrompt
                            while prompt not in ["y","n"]:
                                prompt = input("???replace y/n??")
                        if prompt == "y":
                            needToPatch=True
                else:
                    needToPatch=True
                if needToPatch:
                    #print(MetaData)      
                    MDR=requests.put(url + "samples/"+id+"/meta", headers = headers2,data = MetaData)
                    ####check the MetaData loading was correct
                    if BadRequest(MDR,204):
                        MDR.raise_for_status()
        #print("metadata OK for "+ name + " (" + id + ")")
        ###patch the quantity
        Quant={}
        Note=None
        for fea in ['Amount','Unit']:
            if DICT[sampTy][fea].startswith("fixed"):
                element=DICT[sampTy][fea].split("_")[1]
            elif DICT[sampTy][fea]=="None":
                element="NA"
            else:
                element=TableTYPE[DICT[sampTy][fea]][index]
                if format(element)=="nan":
                    element=0
                elif "<" in format(element):
                    Note="actual amount reported: "+element
                    element=0
            Quant[fea]=element
        Quant["displayUnit"]=Quant["Unit"].capitalize()
        Quant["fullAmount"]=Quant["Amount"]
        QR=requests.put(url + "samples/" + id + "/quantity", headers = headers2, data = Quant)
        if BadRequest(QR,204):
            QR.raise_for_status()
            ###put actual weight in note when there is a "<"

        if Note is not None:
            r=requests.get(url + "samples/"+id, headers = headers2)
            if BadRequest(r,200):
                r.raise_for_status()
            Data=r.json()
            Data["note"]=Data["note"]+" / "+ Note
            r=requests.patch(url + "samples/"+id, headers = headers2,data = Data)
            if BadRequest(r,204):
                r.raise_for_status()
        
print("finished")




CH0001
CH0002
CH0003
CH0004
CH0005
CH0006
CH0007
difference for CH0007(feature: Archaeologist ID) 04Ca012-P-51-2 vs loaded : P-51
CH0008
difference for CH0008(feature: Archaeologist ID) 04Ca012-P-57-1 vs loaded : P-57
CH0009
difference for CH0009(feature: Archaeologist ID) 04Ca012-P-57-2 vs loaded : P-57
CH0010
difference for CH0010(feature: Archaeologist ID) 04Ca012-P-91-1 vs loaded : P-91
CH0011
difference for CH0011(feature: Archaeologist ID) 04Ca012-P-97-1 vs loaded : P-97
CH0012
difference for CH0012(feature: Archaeologist ID) 04Ca012-P-97-2 vs loaded : P-97
CH0013
difference for CH0013(feature: Archaeologist ID) 04Ca012-P-97-4 vs loaded : P-97
CH0014
difference for CH0014(feature: Archaeologist ID) 04Ca012-P-97-3 vs loaded : P-97
CH0015
difference for CH0015(feature: Archaeologist ID) 04Ca012-P-97-5 vs loaded : P-97
CH0016
difference for CH0016(feature: Archaeologist ID) 04Ca012-P-105-5 vs loaded : P-105
CH0017
difference for CH0017(feature: Archaeologist ID) 04Ca012-P-105-7 vs l

## Assignation to storage
first organize the storage IDs (a bit messy but eLab treats all storage levels similarly for sample assignation)

In [ ]:
storageByID={}
r=requests.get(url+"/storageLayers",headers=headers1)
if BadRequest(r,200):
    r.raise_for_status()
    
stoData=r.json().get("data")
for sto in stoData:
    
    storageByID[sto["storageLayerID"]]={"name":sto["name"],"parentID":sto["parentStorageLayerID"]}

def getParentSto(ID,stoDict):
    if stoDict[ID]["parentID"]==0:
        return(stoDict[ID]["name"])
    else:
        return(getParentSto(stoDict[ID]["parentID"],stoDict)+", "+stoDict[ID]["name"])
    
storage={}
storageRev={}
for stoID in storageByID.keys():
    name=getParentSto(stoID,storageByID)
    storage[name]=stoID
    storageRev[stoID]=name
    


### Individual and Site  assignation to Artefactual Storage 


In [ ]:
for sampTy in [ "Skeleton Element", "Individual","Site"]:
    TableTYPE={}
    for fea in DICT[sampTy].keys():
        if DICT[sampTy][fea].startswith("Storage") or fea == "Name":
            TableTYPE[DICT[sampTy][fea]]=Table[DICT[sampTy][fea]]
        
    #TableTYPE=TableTYPE.drop_duplicates()
    TableTYPE=pandas.DataFrame(TableTYPE).drop_duplicates()
    if sampTy+"_Storage" not in TableTYPE.keys():
        TableTYPE[sampTy+"_Storage"]="" 
    if len(TableTYPE.loc[TableTYPE[sampTy+"_Storage"] == "",sampTy+"_Storage"])>0:
        UnspecifiedStorage="?"
        while UnspecifiedStorage not in storage:
            UnspecifiedStorage=input("Storage for "+sampTy+" with missing storage info in input table?")
        TableTYPE.loc[TableTYPE[sampTy+"_Storage"] =="",sampTy+"_Storage"]=UnspecifiedStorage

    for index,name in TableTYPE[DICT[sampTy]['Name']].items():
        if format(name)=="nan":
            continue
        id=format(registered[sampTy][name])
        r=requests.get(url+"/samples/"+id,headers=headers1,data={})
        if BadRequest(r,200):
            r.raise_for_status()
        IDstoAlready=r.json()['storageLayerID']
        IDsto=storage[ TableTYPE[sampTy+"_Storage"][index]]
        prompt=defaultPrompt
        if IDstoAlready != 0:
            if(format(IDstoAlready) != format(IDsto)):
                #if storageRev[IDstoAlready] in ["In Copenhagen","Nico office","In Tartu","Unknown"]:
                #    prompt="n"
                #elif storageRev[IDstoAlready] == "With Lumila":
                #    prompt="y"
                while prompt not in ["y","n"]:
                    prompt=input("move "+name+" from "+storageRev[IDstoAlready]+ " to "+storageRev[IDsto]+" (y/n)?")
        else:
            prompt="y"
        if prompt == "y":
                r=requests.post(url+"/samples/moveToLayer/"+format(IDsto)+"?sampleIDs="+id,headers=headers1,data={})
                if BadRequest(r,204):
                    r.raise_for_status()
print("finished")

## Assign individual and Skeleton Element to Labelling experiment

In [ ]:
r = requests.get(url + "experiments", headers = headers2,params = params)
data = r.json()
experiments = {}
for exp in data.get("data"):
    experiments[format(exp.get("name"))] = format(exp.get("experimentID"))

for expe in ["Labelling process"]:
    #print(expe)
    idExpe=experiments[expe]
    r=requests.get("https://elab-dev.pasteur.fr/api/v1/experiments/"+idExpe+"/sections",headers=headers1)
    if r.status_code != 200:
        print(r.status_code)
        print(r.raise_for_status())
    if r.json().get("recordCount") == 0:
        print("no record")
        continue
    SampleIN={}
    SampleOUT={}
    for data in r.json().get("data"):
        if data["sectionType"] == "SAMPLESIN":
            SampleIN[data["sectionHeader"]]=data["expJournalID"]
        elif data["sectionType"] == "SAMPLESOUT":
            SampleOUT[data["sectionHeader"]]=data["expJournalID"]
    experiments[expe]={"ID":idExpe,
                      "sampleIN":SampleIN,
                      "sampleOUT":SampleOUT}




sampleOUT=experiments["Labelling process"]["sampleOUT"]["Labelled Skeleton elements "]
sampleIN=experiments["Labelling process"]["sampleIN"]["Individuals labelled"]



In [ ]:
listOUT=[]
listIN=[]
for i,inName in Table["Skeleton Element"].items():
    if(format(inName) == "nan"):
        continue
    inID=registered["Skeleton Element"][inName]
    listOUT.append(inID)
    ###get the parent individual
    r=requests.get(url+"/samples/"+inID+"/parent",headers=headers1)
    if BadRequest(r,200):
        r.raise_for_status()
        break
    rjson=r.json()
    outName=rjson.get("name")
    listIN.append(rjson.get("sampleID"))

print(listIN)
print(listOUT)

In [ ]:
r=requests.put(url+"/experiments/sections/"+format(sampleOUT)+"/samples",headers=headers1,data = format(listOUT))
if BadRequest(r,204):
    r.raise_for_status()



In [ ]:
r=requests.put(url+"/experiments/sections/"+format(sampleIN)+"/samples",headers=headers1,data = format(listIN))
if BadRequest(r,204):
    r.raise_for_status()